# Creating a basic chat experience with kernel arguments

In this example, we show how you can build a simple chat bot by sending and updating arguments with your requests. 

We introduce the Kernel Arguments object which in this demo functions similarly as a key-value store that you can use when running the kernel.  

In this chat scenario, as the user talks back and forth with the bot, the arguments get populated with the history of the conversation. During each new run of the kernel, the arguments will be provided to the AI with content. 

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.0.1"
#!import config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;

var builder = Kernel.CreateBuilder();

// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

Installed Packages Microsoft.SemanticKernel, 1.0.1

Let's define a prompt outlining a dialogue chat bot.

In [2]:
const string skPrompt = @"
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$userInput}}
ChatBot:";

var executionSettings = new OpenAIPromptExecutionSettings 
{
    MaxTokens = 2000,
    Temperature = 0.7,
    TopP = 0.5
};

Register your semantic function

In [3]:
var chatFunction = kernel.CreateFunctionFromPrompt(skPrompt, executionSettings);

Initialize your arguments

In [4]:
var history = "";
var arguments = new KernelArguments()
{
    ["history"] = history
};

Chat with the Bot

In [5]:
var userInput = "Hi, I'm looking for book suggestions";
arguments["userInput"] = userInput;

var bot_answer = await chatFunction.InvokeAsync(kernel, arguments);

Update the history with the output and set this as the new input value for the next request

In [6]:
history += $"\nUser: {userInput}\nAI: {bot_answer}\n";
arguments["history"] = history;

Console.WriteLine(history);


User: Hi, I'm looking for book suggestions
AI: Sure, I'd be happy to help with that. Could you please tell me what kind of books you're interested in? For example, are you looking for fiction or non-fiction? Do you prefer a specific genre like mystery, romance, fantasy, or science fiction?



Keep Chatting!

In [7]:
Func<string, Task> Chat = async (string input) => {
    // Save new message in the arguments
    arguments["userInput"] = input;

    // Process the user message and get an answer
    var answer = await chatFunction.InvokeAsync(kernel, arguments);

    // Append the new interaction to the chat history
    var result = $"\nUser: {input}\nAI: {answer}\n";
    history += result;

    arguments["history"] = history;
    
    // Show the response
    Console.WriteLine(result);
};

In [8]:
await Chat("I would like a non-fiction book suggestion about Greece history. Please only list one book.");


User: I would like a non-fiction book suggestion about Greece history. Please only list one book.
AI: I would recommend "The History of the Peloponnesian War" by Thucydides. It's a classic work of ancient Greek history that provides a detailed account of the war between Athens and Sparta in the 5th century BC.



In [9]:
await Chat("that sounds interesting, what are some of the topics I will learn about?");


User: that sounds interesting, what are some of the topics I will learn about?
AI: "The History of the Peloponnesian War" covers a wide range of topics related to ancient Greece. It provides an in-depth look at the political and military strategies of Athens and Sparta, as well as the cultural, social, and economic factors that influenced the war. You'll also learn about key historical figures, battles, and events. Thucydides' analysis of the causes and consequences of the war is considered one of the earliest and most important works of political history.



In [10]:
await Chat("Which topic from the ones you listed do you think most people find interesting?");


User: Which topic from the ones you listed do you think most people find interesting?
AI: Most people tend to find the political and military strategies of Athens and Sparta particularly interesting. These strategies provide a fascinating insight into the tactical thinking of the time and the unique characteristics of each city-state. Moreover, the exploration of key historical figures and their influence on the events of the war often captivates readers.



In [11]:
await Chat("could you list some more books I could read about the topic(s) you mentioned?");


User: could you list some more books I could read about the topic(s) you mentioned?
AI: Absolutely, here are a few more books that delve into the history and politics of ancient Greece:

1. "The Peloponnesian War" by Donald Kagan - This is a modern retelling of the war between Athens and Sparta, providing a comprehensive overview of the entire conflict.

2. "Persian Fire: The First World Empire and the Battle for the West" by Tom Holland - This book provides a look at the Persian Wars, which preceded the Peloponnesian War and greatly influenced the political landscape of Greece.

3. "The Spartans: The World of the Warrior-Heroes of Ancient Greece" by Paul Cartledge - This book offers an in-depth look at Sparta, one of the key players in the Peloponnesian War.

4. "The Rise and Fall of Athens: Nine Greek Lives" by Plutarch - This collection of biographies provides insight into some of the key figures in Athenian history.

5. "Thucydides: The War of the Peloponnesians and the Athenians"

After chatting for a while, we have built a growing history, which we are attaching to each prompt and which contains the full conversation. Let's take a look!

In [12]:
Console.WriteLine(history);


User: Hi, I'm looking for book suggestions
AI: Sure, I'd be happy to help with that. Could you please tell me what kind of books you're interested in? For example, are you looking for fiction or non-fiction? Do you prefer a specific genre like mystery, romance, fantasy, or science fiction?

User: I would like a non-fiction book suggestion about Greece history. Please only list one book.
AI: I would recommend "The History of the Peloponnesian War" by Thucydides. It's a classic work of ancient Greek history that provides a detailed account of the war between Athens and Sparta in the 5th century BC.

User: that sounds interesting, what are some of the topics I will learn about?
AI: "The History of the Peloponnesian War" covers a wide range of topics related to ancient Greece. It provides an in-depth look at the political and military strategies of Athens and Sparta, as well as the cultural, social, and economic factors that influenced the war. You'll also learn about key historical figur